In [15]:
#practice collecting data from imdb actor pages

#import some handy packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import random
import re
import csv

In [16]:
#setup three output files (fbak for storing websites collected, fout for storing data scraped, fout2 for recording pages scraped)
fbak = open('src-pgs.txt', 'ab')
fout = open('imdb-recs.txt', 'a', newline='')
fout2 = open('imdb-pgs.txt', 'a')
#create a handle to write tab-delimited data to the file fout
writer = csv.writer(fout, delimiter='\t')
#send the header row to this handle
#writer.writerow(['movie','movie_url','actor','actor_url','role','role_url','year'])

In [17]:
#read in html from website
#html = urlopen('http://www.imdb.com/title/tt2488496/').read(); fbak.write(html);
#time.sleep(random.randint(4, 10));
#note: sleeping is essential to avoid annoying web servers by calling them too often!

#or copy html into a text file and read it in from there to develop your scraping program first
f = open('actorsrc.txt', 'rb'); html = f.read()

#convert the html into a format that recognizes the structure of the html (and put it into a variable we're calling soup)
soup = BeautifulSoup(html, "html.parser")

In [18]:
#***Code below finds all the variables we want about actors and outputs them to a file handle we're calling fout***

#get actor name, which is string stored under html 'title' tag, & strip out any whitespace at the start/end; then clean out IMDB text
actor = soup.title.string.strip(); actor = actor.encode('utf-8', 'ignore'); actor = actor.decode('utf-8'); actor = re.sub(' - IMDB','',actor, flags=re.IGNORECASE)
#get actor url
tag = soup.find("link", rel="canonical"); actor_url = tag["href"]
#find the tag at the start of the list of acting credits
div = soup.find("div", class_="filmo-category-section")
#find all the credits nested under this tag
credits = div.find_all("div", class_="filmo-row")
print(len(credits),"credits in filmography")

19 credits in filmography


In [19]:
goodct=0
#iterate thru list of credits
for row in credits:
  txt = row.text.strip(); txt = txt.encode('utf-8', 'ignore'); txt = txt.decode('utf-8'); #print(txt)
  if '(TV' in txt:
    continue
  yrsect = row.span.text.strip()
  m = re.search('(\d\d\d\d)', yrsect)
  if m:
    year = m.group(1); year=int(year)
  else:
    year = ''
  links = row.find_all("a")
  ct=len(links)
  if ct != 2:
    continue
  #iterate thru columns to store actors and roles
  ctr=0
  for link in links:
    txt = link.text.strip(); txt = txt.encode('utf-8', 'ignore'); txt = txt.decode('utf-8'); #print(txt)
    url = link["href"]; url = re.sub('\?ref.+','',url); url='http://www.imdb.com' + url;
    ctr+=1
    if ctr==1:
      movie_url=url; movie=txt
    else:
      role_url=url; role=txt
  rec = [movie,movie_url,actor,actor_url,role,role_url,year]
  #send our list to the output handle
  writer.writerow(rec); goodct+=1

In [20]:
#output url of page we have scraped
fout2.write(actor_url + "\n")
#report how many good rows were found
print('parsed',actor)
print('credits found:',len(credits),'  and movies output:',goodct)

#close file handles
fout.close(); fout2.close(); fbak.close()

parsed Gal Gadot
credits found: 19   and movies output: 11
